In [ ]:
!unzip -q '/content/drive/MyDrive/oto_recommend/data/otto-validation.zip'
!kaggle kernels output luckyzq/otto-tuning-pipeline2-lb-0-577 -p './'
!kaggle datasets download -d alexlods/ottoprecalculatedfeatureparquet
!unzip -q /content/ottoprecalculatedfeatureparquet.zip

Output file downloaded to ./__notebook_source__.ipynb
Output file downloaded to ./top_50_buy2buy_fortrain_v7_0.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_0.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_1.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_2.pqt
Output file downloaded to ./top_50_carts_orders_fortrain_v7_3.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_0.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_1.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_2.pqt
Output file downloaded to ./top_50_clicks_fortrain_v7_3.pqt
Kernel log downloaded to ./otto-tuning-pipeline2-lb-0-577.log 
 98% 465M/475M [00:04<00:00, 144MB/s]
100% 475M/475M [00:04<00:00, 111MB/s]


In [ ]:
import pandas as pd
import glob
import datetime,time
from tqdm.notebook import tqdm
from  datetime  import datetime
!pip install -q pandarallel
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import gc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
def get_train_data():
  result = {}
  objects = ['clicks','carts','orders']
  for t in objects:
    if t == 'clicks':
      df_train = pd.read_parquet(f'/content/drive/MyDrive/oto_recommend/process_data/train_{t}_v2.parquet')
      df_train['rank'] =  df_train.groupby('session')['recall_aids'].cumcount()
      df_negative = df_train[df_train['label'] == 0].sample(int(len(df_train[df_train['label'] == 1])*2),random_state=19990829)
      df_train = pd.concat([df_train[df_train['label']==1],df_negative],axis=0).reset_index(drop=True)
    else:
      df_train = pd.read_parquet(f'/content/drive/MyDrive/oto_recommend/process_data/train_{t}_v2.parquet')
      df_train['rank'] =  df_train.groupby('session')['recall_aids'].cumcount()
      df_negative = df_train[df_train['label'] == 0].sample(int(len(df_train[df_train['label'] == 1])*2),random_state=19990829)
      df_train = pd.concat([df_train[df_train['label']==1],df_negative],axis=0).reset_index(drop=True)
    del df_negative
    gc.collect()
    result[t] = df_train
  return result

In [ ]:
df_train_dict = get_train_data()

In [ ]:
## 1.关联统计特征

In [ ]:
features_stastics = pd.read_parquet('/content/drive/MyDrive/oto_recommend/process_data/feautres_statistic.parquet')
for t in objects:
  df_train_dict[t] = df_train_dict[t].merge(features_stastics,left_on=['recall_aids'],right_on=['aid'],how='left')
del features_stastics
gc.collect()

280

In [ ]:
## 2.关联真实行为中最后的3个ITEM

In [ ]:
## 真实的行为数据
files = glob.glob('/content/test_parquet/*')
for i in tqdm(range(len(files))):
  if i == 0:
    df_test = pd.read_parquet(files[i])
  else:
    df_test = pd.concat([df_test,pd.read_parquet(files[i])],axis=0)
df_test = df_test.reset_index(drop=True)

100%|██████████| 20/20 [00:01<00:00, 10.11it/s]


In [ ]:
for t in objects:
  for t1 in objects:
    tmp1 = df_test[df_test['type'] == t1]
    tmp1['rk'] = tmp1.groupby('session')['ts'].rank(ascending=False,method='first')
    if t1 == 'clicks':
      rk = 3
    else:
      rk = 1
    tmp1 = tmp1[tmp1['rk'] <= rk]
    tmp1 = tmp1.groupby('session')['aid'].progress_apply(lambda x : ','.join(map(str,list(set(x.values)))))
    df_train_dict[t] = df_train_dict[t].merge(tmp1.reset_index().rename(columns={'aid':f'last_{t1}_aid_list'}),on=['session'],how='left')
    if t1 == 'clicks':
      df_train_dict[t] = pd.concat([df_train_dict[t],df_train_dict[t][f'last_{t1}_aid_list'].str.split(',',expand=True).rename(columns={0:f'last_{t1}_3_aid',1:f'last_{t1}_2_aid',2:f'last_{t1}_1_aid'})],axis=1)
    else:
      df_train_dict[t] = pd.concat([df_train_dict[t],df_train_dict[t][f'last_{t1}_aid_list'].str.split(',',expand=True).rename(columns={0:f'last_{t1}_1_aid'})],axis=1)

<ipython-input-35-9043d87531fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['rk'] = tmp1.groupby('session')['ts'].rank(ascending=False,method='first')
100%|██████████| 39635/39635 [00:00<00:00, 55729.64it/s]


In [ ]:
## 3.计算特征之间的共现程度

In [ ]:
VER = 7
DISK_PIECES = 4

top_20_clicks = pd.read_parquet(f'./top_50_clicks_fortrain_v{VER}_0.pqt')

for k in tqdm(range(1,DISK_PIECES)): 
    top_20_clicks = pd.concat([top_20_clicks,pd.read_parquet(f'./top_50_clicks_fortrain_v{VER}_{k}.pqt')],axis=0).reset_index(drop=True)


top_20_buys =  pd.read_parquet(f'./top_50_carts_orders_fortrain_v{VER}_0.pqt') 

for k in tqdm(range(1,DISK_PIECES)): 
    top_20_buys =  pd.concat([top_20_buys,pd.read_parquet(f'./top_50_carts_orders_fortrain_v{VER}_{k}.pqt')],axis=0).reset_index(drop=True)

top_20_buy2buy =  pd.read_parquet(f'./top_50_buy2buy_fortrain_v{VER}_0.pqt') 

100%|██████████| 3/3 [00:05<00:00,  1.88s/it]


In [ ]:
last_aid_features = ['last_clicks_3_aid','last_clicks_2_aid','last_clicks_1_aid','last_carts_1_aid','last_orders_1_aid']
for t in objects:
  df_train_dict[t][last_aid_features] = df_train_dict[t][last_aid_features].fillna(-1).astype('int32')
  for feature in last_aid_features:
    if 'clicks' in feature:
      df_train_dict[t] = df_train_dict[t].merge(top_20_clicks.rename(columns={'aid_x':feature,'aid_y':'recall_aids'}),on=[feature,'recall_aids'],how='left').rename(columns={'wgt':f'features4_{feature}_wgt'})
    elif 'carts' in feature:
      df_train_dict[t] = df_train_dict[t].merge(top_20_buys.rename(columns={'aid_x':feature,'aid_y':'recall_aids'}),on=[feature,'recall_aids'],how='left').rename(columns={'wgt':f'features4_{feature}_wgt'})
    elif 'orders' in feature:
      df_train_dict[t] = df_train_dict[t].merge(top_20_buy2buy.rename(columns={'aid_x':feature,'aid_y':'recall_aids'}),on=[feature,'recall_aids'],how='left').rename(columns={'wgt':f'features4_{feature}_wgt'})

In [ ]:
del top_20_clicks , top_20_buys , top_20_buy2buy
gc.collect()

19

In [ ]:
## 4.session 特征

In [ ]:
## 关联真实的行为日志
df_test_clicks = df_test[df_test['type'] == 'clicks'].groupby('session')['aid'].apply(lambda x : list(set(x))).reset_index()
df_test_carts = df_test[df_test['type'] == 'carts'].groupby('session')['aid'].apply(lambda x : list(set(x))).reset_index()
df_test_orders = df_test[df_test['type'] == 'orders'].groupby('session')['aid'].apply(lambda x : list(set(x))).reset_index()
df_test_clicks.columns = ['session','truth_aid_clicks_list']
df_test_carts.columns = ['session','truth_aid_carts_list']
df_test_orders.columns = ['session','truth_aid_orders_list']

In [ ]:
for t in objects:
  df_train_dict[t] = df_train_dict[t].merge(df_test_clicks,on=['session'],how='left').merge(df_test_carts,on=['session'],how='left').merge(df_test_orders,on=['session'],how='left')

In [ ]:
del df_test_clicks , df_test_carts ,df_test_orders
gc.collect()

0

In [ ]:
for t in objects:
  tmp = df_train_dict[t]
  tmp['features5_clicks_len'] = tmp['truth_aid_clicks_list'].apply(lambda x : len(x) if type(x) == list else 0)
  tmp['features5_carts_len'] = tmp['truth_aid_carts_list'].apply(lambda x : len(x) if type(x) == list else 0)
  tmp['features5_orders_len'] = tmp['truth_aid_orders_list'].apply(lambda x : len(x) if type(x) == list else 0)
  tmp['features5_session_len'] = tmp['features5_clicks_len'] + tmp['features5_carts_len'] + tmp['features5_orders_len']
  df_train_dict[t] = tmp

In [ ]:
## 5.ts特征

In [ ]:
dts = pd.to_datetime(df_test['ts']/1000,unit='s')
df_test['day'] = dts.dt.weekday
df_test['hour'] = dts.dt.hour
df_test['hm'] = (dts.dt.hour*100 + dts.dt.minute*100//60).astype('int16')
df_test_day = df_test.groupby('session')['day'].mean().reset_index()
df_test_hour = df_test.groupby('session')['hour'].mean().reset_index()
df_test_hm = df_test.groupby('session')['hm'].mean().reset_index()

In [ ]:
for t in objects:
  df_train_dict[t] = df_train_dict[t].merge(df_test_day.reset_index(),on=['session'],how='left').merge(df_test_hour.reset_index(),on=['session'],how='left').merge(df_test_hm.reset_index(),on=['session'],how='left')
  df_train_dict[t] = df_train_dict[t].rename(columns={'day':'features6_day','hour':'features6_hour','hm':'features6_hm'})

In [ ]:
del df_test_day , df_test_hour ,df_test_hm
gc.collect()

0

In [ ]:
## 6. embedding特征

In [ ]:
embedding_weight = np.load('/content/word2vec.model.wv.vectors.npy')
aid_embedding = pd.DataFrame(embedding_weight, columns = ['featuress8_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})

In [ ]:
for t in objects:
  df_train_dict[t] = df_train_dict[t].merge(aid_embedding,how='left',left_on = ['recall_aids'] , right_on=['aid'])

In [ ]:
last_click_embedding =  pd.DataFrame(embedding_weight, columns = ['featuress9_last_click_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})
last_carts_embedding =  pd.DataFrame(embedding_weight, columns = ['featuress9_last_carts_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})
last_orders_embedding =  pd.DataFrame(embedding_weight, columns = ['featuress9_last_orders_Embedding_' + str(x) for x in range(32)]).reset_index().rename(columns = {'index':'aid'})
for t in objects:
  df_train_dict[t] = df_train_dict[t].merge(last_click_embedding,how='left',left_on = ['last_clicks_3_aid'] , right_on=['aid'])
  # df_train_dict[t] = df_train_dict[t].merge(last_carts_embedding,how='left',left_on = ['last_carts_1_aid'] , right_on=['aid'])
  # df_train_dict[t] = df_train_dict[t].merge(last_orders_embedding,how='left',left_on = ['last_orders_1_aid'] , right_on=['aid'])

In [ ]:
## 保存训练数据

In [ ]:
for t in objects:
  df_train_dict[t] = df_train_dict[t].drop(columns=['aid_x','aid_y','index_x','index_y','aid'])

In [ ]:
df_train_dict['clicks'].to_parquet('/content/drive/MyDrive/oto_recommend/process_data/train_data_clicks_v5.parquet')
df_train_dict['carts'].to_parquet('/content/drive/MyDrive/oto_recommend/process_data/train_data_carts_v5.parquet')
df_train_dict['orders'].to_parquet('/content/drive/MyDrive/oto_recommend/process_data/train_data_orders_v5.parquet')

In [ ]:
df_train_dict['clicks']

,session,recall_aids,hits_aids,label,rank,all_clicks_count,all_carts_count,all_orders_count,features2_clicks_0,features2_clicks_1,...,featuress9_last_click_Embedding_22,featuress9_last_click_Embedding_23,featuress9_last_click_Embedding_24,featuress9_last_click_Embedding_25,featuress9_last_click_Embedding_26,featuress9_last_click_Embedding_27,featuress9_last_click_Embedding_28,featuress9_last_click_Embedding_29,featuress9_last_click_Embedding_30,featuress9_last_click_Embedding_31
0,11098529,1105029,1105029.0,1,0,195.0,5.0,5.0,23.0,23.0,...,0.396338,-0.241744,0.085979,-0.098529,-0.288573,0.064683,-0.011770,-0.100634,-0.069559,0.112903
1,11098534,908024,908024.0,1,0,1086.0,17.0,17.0,211.0,118.0,...,0.269202,-0.133638,-0.113561,-0.072093,-0.149152,-0.019875,0.021804,-0.014138,-0.219288,-0.122309
2,11098535,745365,745365.0,1,0,59.0,2.0,2.0,6.0,9.0,...,0.591781,-0.451523,0.399852,-0.107120,-0.508324,0.364119,-0.144704,0.041103,0.014857,0.295920
3,11098537,1503532,1503532.0,1,32,1456.0,318.0,318.0,249.0,218.0,...,0.767515,-1.478991,0.219363,-0.069850,-0.101981,1.050852,-0.659992,-0.172946,-0.776353,-0.114598
4,11098538,1452081,1452081.0,1,14,1085.0,61.0,61.0,136.0,150.0,...,0.640347,-0.498484,0.278461,0.021448,-0.396591,0.330053,-0.190134,0.319457,-0.139158,0.017569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040534,12325228,1349119,NaN,0,9,936.0,38.0,38.0,132.0,171.0,...,0.301156,-0.190866,0.158229,-0.063146,-0.085625,0.109423,0.005426,-0.149045,-0.064662,0.064503
3040535,12079983,8479,NaN,0,11,689.0,116.0,116.0,110.0,94.0,...,0.128659,-0.013195,-0.053778,-0.082995,-0.213705,0.023291,0.036499,-0.183359,-0.015874,0.099328
3040536,11849866,865135,NaN,0,31,1027.0,142.0,142.0,111.0,151.0,...,1.054251,-1.285135,-0.218098,-1.187781,-0.148343,-0.052817,0.385011,-1.490051,-2.891479,0.603718
3040537,11451504,604085,NaN,0,0,4161.0,188.0,188.0,443.0,504.0,...,0.328452,0.082732,-0.316664,-0.488054,-0.860623,0.146916,0.181655,-0.772222,-0.070572,-0.405018
